### Imports

In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from time import time
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='talk', fscale=1.8, spines=False, gridlines='--', ticks=True, grid=False, figsize=(12, 8))

### Load the data & build the product-transaction matrix

In [2]:
def get_transaction_data():
    """Load groceries transaction data into DataFrame"""
    df = pd.read_csv('grocery_transactions.csv')
    df = df.stack().reset_index(-1, drop=True)
    df.index.names = ['tx_id']
    df = pd.get_dummies(df, prefix='', prefix_sep='')
    return df.groupby(level='tx_id').sum()

### Create itemset candidates

In [11]:
data = get_transaction_data()

item_id = pd.Series(dict(enumerate(data.columns)))
print(item_id.head())
transactions = data.values
print(transactions.shape)

min_support = 0.01
item_length = 1
candidates = list(zip(item_id.index))
candidates_tested = 0
itemsets = pd.DataFrame(columns=['support', 'length'])

0    Instant food products
1                 UHT-milk
2         abrasive cleaner
3         artif. sweetener
4           baby cosmetics
dtype: object
(9834, 169)


In [4]:
candidates[:10]

[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,)]

### Candidate Generation

In [5]:
for i in range(1, 5):
    remaining_items = np.unique([item for t in candidates for item in t])
    new_candidates = list(combinations(remaining_items, r=i))
    print('Length {}: {:>10,.0f}'.format(i, len(new_candidates)), 
          new_candidates[:5])

Length 1:        169 [(0,), (1,), (2,), (3,), (4,)]
Length 2:     14,196 [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]
Length 3:    790,244 [(0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 1, 6)]
Length 4: 32,795,126 [(0, 1, 2, 3), (0, 1, 2, 4), (0, 1, 2, 5), (0, 1, 2, 6), (0, 1, 2, 7)]


### The apriori pruning based on support

In [6]:
def prune_candidates(all_txn, candidates, candidate_size, min_support):
    """Return DataFrame with itemsets of candidate_size with min_support
        all_txn: numpy array of transaction-product matrix
        candidates: list of tuples containing product id
        candidate_size: length of item set
        min_support: support threshold
        """
    itemsets = {}
    for candidate in candidates:
        candidate_txn = all_txn[:, candidate].reshape(-1, candidate_size)  
        relevant_txn = candidate_txn[(candidate_txn == 1).all(axis=1)]
        candidate_support = relevant_txn.shape[0] / all_txn.shape[0]
        if candidate_support >= min_support:
            itemsets[frozenset(candidate)] = candidate_support
    result = pd.Series(itemsets).to_frame('support')
    return result.assign(length=candidate_size) 

### Running the apriori algorithm

In [7]:
while candidates:
    new_items = prune_candidates(
        transactions, candidates, item_length, min_support)
    itemsets = itemsets.append(new_items)
    candidates_tested += len(candidates)
    print('Itemset Length {}\tCandidates: {:>7,.0f}\tNew Items: {:>7,.0f}'
          .format(item_length, len(candidates), len(new_items)))
    item_length += 1
    remaining_items = np.unique([item for t in new_items.index for item in t])
    candidates = list(combinations(remaining_items, r=item_length))
print('\nPotential Itemsets: {:,.0f} \nTested Itemsets: {:,.0f}'.format(
    2**len(item_id) - 1, candidates_tested))

Itemset Length 1	Candidates:     169	New Items:      88
Itemset Length 2	Candidates:   3,828	New Items:     213
Itemset Length 3	Candidates:  16,215	New Items:      32
Itemset Length 4	Candidates:   3,060	New Items:       0

Potential Itemsets: 748,288,838,313,422,294,120,286,634,350,736,906,063,837,462,003,712 
Tested Itemsets: 23,272


In [8]:
itemsets.info()
itemsets.sort_values('support', ascending=False).head()

<class 'pandas.core.frame.DataFrame'>
Index: 333 entries, (1) to (162, 166, 167)
Data columns (total 2 columns):
support    333 non-null float64
length     333 non-null object
dtypes: float64(1), object(1)
memory usage: 7.8+ KB


,support,length
(166),0.255542,1
(103),0.193512,1
(123),0.183954,1
(139),0.174395,1
(167),0.139516,1


In [9]:
itemsets.length.value_counts().sort_index()

1     88
2    213
3     32
Name: length, dtype: int64

In [10]:
itemsets.support.describe()

count    333.000000
mean       0.025071
std        0.027325
min        0.010067
25%        0.011897
50%        0.016270
75%        0.026032
max        0.255542
Name: support, dtype: float64